EFFETS DU COVID SUR LECONOMIE DE LA ZONE EURO

Les pays de la Zone Euro étudiés sont ceux présents avant le 01/01/2023 :
Austria, Belgium, Cyprus, Estonia, Finland, France, Germany, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Portugal, Slovakia, Slovenia, Spain, Greece

In [1]:
# Librairies

import pandas as pd
import gzip
import numpy as np
import seaborn as sn
from datetime import datetime
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

In [2]:
# Listes des pays de la zone euro (avant 2023) 

euro_area_19 = ['Austria', 'Belgium', 'Cyprus', 'Estonia', 'Finland', 'France', 'Germany', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Greece']
euro_area_19_ISO = ['AT', 'BE', 'CY', 'EE', 'FI', 'FR', 'DE', 'IE', 'IT', 'LV', 'LT', 'LU','MT', 'NL', 'PT', 'SK', 'SI', 'ES', 'GR' , 'EL']

In [5]:
# Extractions des données

chemin_fichier_covid = '../Data/Data 2/owid-covid-data.csv'
chemin_europe = '../Data/Donnees_Europe/'

cols_BD = ['geo', 'TIME_PERIOD', 'OBS_VALUE']
cols_covid_eur = ['iso_code', 'continent', 'location', 'date', 'total_cases_per_million', 'new_cases_per_million', 'reproduction_rate', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'stringency_index', 'hospital_beds_per_thousand', 'human_development_index']
cols_pop = ['TIME_PERIOD', 'OBS_VALUE']

hdi_Europe = pd.read_csv(chemin_europe+'Europe_human-development-index.csv')
bd_hdi_eur = hdi_Europe.loc[hdi_Europe.Entity.isin(euro_area_19)]

expenditure_travels_Europe = pd.read_csv(chemin_europe+'Europe_expenditure_travels.csv.gz', compression='gzip', sep = ',')
bd_expenditure_travels_eur = expenditure_travels_Europe.loc[expenditure_travels_Europe.geo == 'EA19']

inflation_Europe = pd.read_csv(chemin_europe+'Europe_inflation.csv.gz', compression='gzip', sep = ',')
bd_inflation_eur = inflation_Europe.loc[inflation_Europe.geo == 'EA19']

# -----------------------------------------------------------

population_Europe = pd.read_csv(chemin_europe+'Europe_Population.csv', sep = ',', skiprows=5, usecols=[0,1])
population_Europe.columns = ['TIME_PERIOD', 'OBS_VALUE']

bd_population_eur = pd.DataFrame(columns = cols_pop)
bd_population_eur.TIME_PERIOD = population_Europe.TIME_PERIOD.loc[population_Europe.TIME_PERIOD >= 2017]
bd_population_eur.OBS_VALUE = population_Europe.OBS_VALUE.loc[population_Europe.TIME_PERIOD >= 2017]

del population_Europe

# -----------------------------------------------------------

covid_Europe = pd.read_csv(chemin_fichier_covid, usecols= cols_covid_eur)

bd_covid_eur = covid_Europe.loc[covid_Europe.location.isin(euro_area_19)]
bd_covid_eur = bd_covid_eur[cols_covid_eur]
bd_covid_eur['date'] = pd.to_datetime(bd_covid_eur['date'])
bd_covid_eur['Quarter'] = bd_covid_eur['date']
bd_covid_eur['Quarter'] = pd.PeriodIndex(bd_covid_eur['Quarter'], freq='Q').to_timestamp()

del covid_Europe

# -----------------------------------------------------------

gdp_Europe = pd.read_csv(chemin_europe+'Europe_GDP.csv.gz', compression='gzip', sep = ',')
bd_gdp_eur = gdp_Europe.loc[gdp_Europe.geo == 'EA19']

chomage_Europe = pd.read_csv(chemin_europe+'Europe_Chomage.csv.gz', compression='gzip', sep = ',')
bd_chomage_eur = chomage_Europe.loc[chomage_Europe.geo == 'EA19']
bd_chomage_eur = bd_chomage_eur.loc[chomage_Europe.unit == 'PC_POP']

revenus_foyers_Europe = pd.read_csv(chemin_europe+'Europe_Revenus_foyers.csv.gz', compression='gzip', sep = ',')
bd_revenus_foyers_eur = revenus_foyers_Europe.loc[revenus_foyers_Europe.geo == 'EA19'] 

bd_gdp_eur['TIME_PERIOD'] = list(pd.PeriodIndex(bd_gdp_eur.TIME_PERIOD, freq='Q')).to_timestamp()
bd_chomage_eur['TIME_PERIOD'] = list(pd.PeriodIndex(bd_chomage_eur.TIME_PERIOD, freq='Q')).to_timestamp()
bd_revenus_foyers_eur['TIME_PERIOD'] = list(pd.PeriodIndex(bd_revenus_foyers_eur.TIME_PERIOD, freq='Q')).to_timestamp()

gdp_q = bd_gdp_eur.loc[pd.Period('2019Q1') <= bd_gdp_eur.TIME_PERIOD]
gdp_q = gdp_q.loc[pd.Period('2022Q3') >= gdp_q.TIME_PERIOD]

chom_q = bd_chomage_eur.loc[pd.Period('2019Q1') <= bd_chomage_eur.TIME_PERIOD]
chom_q = chom_q.loc[pd.Period('2022Q3') >= chom_q.TIME_PERIOD]

revenus_q = bd_revenus_foyers_eur.loc[pd.Period('2019Q1') <= bd_revenus_foyers_eur.TIME_PERIOD]
revenus_q = revenus_q.loc[pd.Period('2022Q3') >= revenus_q.TIME_PERIOD]

bd_quarters = pd.DataFrame(columns = ['Quarter', 'GDP', 'Chomage', 'Revenus'])

bd_quarters['GDP'] = list(gdp_q.OBS_VALUE)
bd_quarters['Chomage'] = list(chom_q.OBS_VALUE)
bd_quarters['Revenus'] = list(revenus_q.OBS_VALUE)
bd_quarters['Quarter'] = list(gdp_q.TIME_PERIOD)

del chomage_Europe, gdp_Europe, revenus_foyers_Europe
del bd_chomage_eur, bd_gdp_eur, bd_revenus_foyers_eur
del gdp_q, chom_q, revenus_q

# household_elec_prices_Europe = pd.read_csv(chemin_europe+'Europe_household_elec_prices.csv.gz', compression='gzip', sep = ',')
# air_passengers_transport_Europe = pd.read_csv(chemin_europe+'Europe_air_passengers_transport.csv.gz', compression='gzip', sep = ',')
# freight_transport_Europe = pd.read_csv(chemin_europe+'Europe_freight_transport.csv.gz', compression='gzip', sep = ',')
# non_household_elec_prices_Europe = pd.read_csv(chemin_europe+'Europe_non_household_elec_prices.csv.gz', compression='gzip', sep = ',')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Donnees_Europe/Europe_human-development-index.csv'

# 

In [ ]:
#